In [3]:
import pandas as pd
import win32com.client as win32
import time
import requests
import json
import openpyxl
import pandas_datareader.data as web
from datetime import date, timedelta
from openpyxl import load_workbook
import yfinance as yf
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import shutil

Puxando posição detalhada c/ selenium (webscrapping)

In [4]:

options = webdriver.ChromeOptions()
prefs = {"download.default_directory" : r"C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\posicao_detalhada"}
#example: prefs = {"download.default_directory" : "C:\Tutorial\down"};
options.add_experimental_option("prefs",prefs)

navegador = webdriver.Chrome(options=options)
navegador.get('https://portal.xpi.com.br/')
id_cliente= '2563257'
navegador.find_element(By.XPATH, '//*[@id="txtLogin"]').send_keys(id_cliente)
sleep(2)
navegador.find_element(By.XPATH, '//*[@id="btnOkLogin"]').click()
sleep(25)
navegador.find_element(By.XPATH, '//*[@id="root"]/div/div/soma-tabs/soma-tab[2]/a').click()
sleep(2)
navegador.find_element(By.XPATH, '//*[@id="root"]/arsenal-loader/div/div[2]/div[2]/div[1]/div/div/div[4]/div/soma-button[1]').click()
sleep(2)
navegador.close()
sleep(4)
src = r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\posicao_detalhada'
dst = r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos'
filelist = ['PosicaoDetalhada']

files = os.listdir( src )
for filename in files:
    filelist.append(filename)
    fullpath = src + '/' + filename
    shutil.move(os.path.join(src, filename), os.path.join(dst, filename))

Puxar Valor Dolar

In [5]:
requisicao = requests.get(r'https://economia.awesomeapi.com.br/all/USD-BRL')
cotacao = requisicao.json()
cotacao_dolar = float(cotacao['USD']['bid'])
print(cotacao_dolar)

5.4418


Configurando dias

In [6]:
today = date.today()
start_day = today - timedelta(days = 7)


Puxar quantidade Dolar existente

Atualizar planilha com bolsa Down Jones

In [7]:

carteirausa_df = pd.read_excel(
    r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\Carteira_usa.xlsx', index_col=0)
# display(carteirausa_df)
tickers_DowJones = "AMZN BRK-A GOOGL LAND MSFT PG SBUX WPC TGT"
datayf = yf.download(tickers_DowJones, start=start_day, end=today)
datayf = datayf['Adj Close']
# display(datayf)
for i in datayf:
    x = datayf[i][0]
    #print(x)
    carteirausa_df[i]['ValorDol'] = x
    #print(carteirausa_df['AMZN']['ValorDol'])
    carteirausa_df[i]['ValorDol'] = carteirausa_df[i]['ValorDol'] * carteirausa_df[i]['Qtde']
    carteirausa_df[i]['ValorReal'] = carteirausa_df[i]['ValorDol'] * cotacao_dolar
carteirausa_df['Total'] = carteirausa_df.sum(axis=1)
display(carteirausa_df)
dolartt = carteirausa_df['Total']['ValorReal']
print(dolartt)
carteirausa_df.to_excel(r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\Carteira_usa_dia.xlsx')


[*********************100%***********************]  9 of 9 completed


,AMZN,BRK-A,GOOGL,LAND,MSFT,PG,SBUX,WPC,TGT,Total
Ativos,,,,,,,,,,
Qtde,16.123900,0.004661,17.590000,61.152800,7.480000,14.325700,24.012800,17.979000,16.966275,175.635136
ValorDol,1319.257493,2143.127800,1513.091741,1103.196568,1754.284391,2176.933468,2367.662043,1393.642174,2473.683002,16244.878681
ValorReal,7179.135426,11662.472862,8233.942636,6003.375084,9546.464798,11846.436547,12884.343308,7583.921983,13461.288161,88401.380804


88401.38080427841


Puxando montante em criptos

In [8]:
carteira_cripto_df = pd.read_excel(
    r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\Carteira_cripto.xlsx', index_col=0)
# display(carteira_cripto_df)
tickers_Cripto = "BTC-USD ETH-USD DOGE-USD"
datayf = yf.download(tickers_Cripto, start="2022-12-22", end="2022-12-23")
datayf = datayf['Adj Close']
# display(datayf)
for i in datayf:
    x = datayf[i][0]
    #print(x)
    carteira_cripto_df[i]['ValorDol'] = x
    #print(carteira_cripto_df['AMZN']['ValorDol'])
    carteira_cripto_df[i]['ValorDol'] = carteira_cripto_df[i]['ValorDol'] * carteira_cripto_df[i]['Qtde']
    carteira_cripto_df[i]['ValorReal'] = carteira_cripto_df[i]['ValorDol'] * cotacao_dolar
carteira_cripto_df['Total'] = carteira_cripto_df.sum(axis=1)
display(carteira_cripto_df)
CriptoTotal = carteira_cripto_df['Total']['ValorReal']
print(CriptoTotal)
carteira_cripto_df.to_excel(r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\Carteira_criptos_dia.xlsx')


[*********************100%***********************]  3 of 3 completed


,BTC-USD,ETH-USD,DOGE-USD,Total
Ativos,,,,
Qtde,0.027102,0.055097,2486.104852,2486.187051
ValorDol,456.134409,67.118778,191.348035,714.601221
ValorReal,2482.192225,365.246964,1041.277734,3888.716924


3888.716923624794


Abrindo Data Frames

Abrir Info.xls e alterar o valor do dolar cotação e dolar montante total

In [9]:

wb = load_workbook(filename=r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\infos.xlsx')
ws = wb.worksheets[1]
ws['I4'] = cotacao_dolar
ws['I2'] = dolartt
ws['H2'] = CriptoTotal
wb.save(r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\infos.xlsx')


Excel file, open, refresh and close

In [10]:
# Start an instance of Excel
xlapp = win32.DispatchEx("Excel.Application")

# Open the workbook in said instance of Excel
wb = xlapp.workbooks.open(r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\infos.xlsx')

# Optional, e.g. if you want to debug
xlapp.Visible = True

# Refresh all data connections.
wb.RefreshAll()
wb.Save()

# Quit
xlapp.Quit()
time.sleep(5)

In [ ]:
# filenames
excel_names = [r"C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\base_dash.xlsx", r"C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\infos.xlsx"]

# read them in
excels = [pd.ExcelFile(name) for name in excel_names]

# turn them into dataframes
frames = [x.parse(x.sheet_names[0], header=None,index_col=None) for x in excels]

# delete the first row for all frames except the first
# i.e. remove the header row -- assumes it's the first
frames[1:] = [df[1:] for df in frames[1:]]

# concatenate them..
combined = pd.concat(frames)

# write it out
combined.to_excel("base_dash.xlsx", header=False, index=False)

In [ ]:
os.startfile(r'"C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\DashFinanceiro_PBI.pbix"')